In [1]:
import pandas as pd

In [2]:
import matplotlib.pyplot as plt
import seaborn as sbs

In [3]:
#leemos archivo con los tickets de otrs
otrs = pd.read_excel("C:\curso_python\pandas\otrs_csv.xlsx")

In [4]:
#renombramos la columna "Downtime ( Min )" por "Downtime"
otrs = otrs.rename(
    columns = {
        "DownTime ( Min )" : "Downtime"
    }
)

In [5]:
#creando downtime de track
track_downtime = otrs[otrs["Queue"].str.contains("Track3")].groupby("Queue",as_index=False)["Downtime"].sum().sort_values(by="Downtime",ascending = False).loc[lambda x : x["Downtime"] > 0]

In [6]:
#creando downtime de backoffice
backoffice_downtime = otrs[otrs["Queue"].str.contains("BackOffice")].groupby("Queue",as_index=False)["Downtime"].sum().sort_values(by="Downtime",ascending=False).loc[lambda x: x["Downtime"] > 0]

In [7]:
#creando downtime de operaciones
ncd_downtime = otrs[otrs["Queue"].str.contains("Operaciones Coto Digital")].groupby("Queue",as_index=False)["Downtime"].sum().sort_values(by="Downtime",ascending=False).loc[lambda x:x["Downtime"]>0]

In [9]:

# Función para obtener el valor antes del ":"
def obtener_valor(texto):
    partes  = texto.split(":")
    return partes[0] if len(partes) > 0 else ""

# Aplicar la función a la columna "Queue" y crear una nueva columna "Cola Principal"
otrs['Cola Principal'] = otrs['Queue'].apply(obtener_valor)

In [10]:
#cambio de nombre la columna Created a Fecha
otrs = otrs.rename (
    columns= {
        "Created" : "Fecha"
    }
)

In [11]:
#separo la columna para solamente tener la fecha sin la hora ni la region
#se dividen las columnas y obtengo la posicion 0
otrs["Fecha"] = otrs["Fecha"].str.split(' ').str[0]
otrs["Fecha"]= pd.to_datetime(otrs["Fecha"])

In [12]:
downtime_general = otrs.groupby("Cola Principal",as_index=False)["Downtime"].sum().sort_values(by="Downtime",ascending=False)

In [13]:
#agregar columna con la cantidad de tickets de cada cola
#agregar la suma de tiempo trabajado de cada cola
#separar el alcance que tuvo cada cola

In [14]:
downtime_general

,Cola Principal,Downtime
7,Multistore,1023.0
8,Operaciones Coto Digital,918.0
2,BackOffice,443.0
11,Track3,435.0
5,Comunicaciones,242.0
10,Sitio,210.0
3,Base de Datos,46.0
9,Seguridad de la Informacion,45.0
0,AIX,0.0
1,Administracion NT,0.0
